# Executive Summary

In [ ]:
# This analysis looks at the frequency of flights on radar tracks at KHPN. The question being reviewed is "Has the frequency of flights changed?"

In [ ]:
# Data used here is 3D radar track data from August 2005, 2010, 2015, and 2018

# Import Required Packages

In [ ]:
from __future__ import print_function
import json
import time
import datetime
from urllib.request import urlopen

import os
import math
import matplotlib
import pandas as pd
import io

import numpy as np
import numpy.random
import matplotlib.pyplot as plt
from matplotlib import cm as CM
from matplotlib import pyplot as PLT

from bs4 import BeautifulSoup

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Define Functions

In [ ]:
def trackScope(df,southernLat,northernLat,westernLong,easternLong,maxAlt):
    df = df[(df['Latitude']>=southernLat) & (df['Latitude']<=northernLat) & (df['Longitude']<=easternLong) & (df['Longitude']>=westernLong)] 
    return df

In [ ]:
def trackPrep(df):
    df['Actual Date/Time'] = pd.to_datetime(df['Actual Date/Time'])
    df['Track Point Time'] = pd.to_datetime(df['Track Point Time'])
    return df

In [ ]:
def trackDiffs(df):
    df = df.groupby('Operation No')['Track Point Time'].min().reset_index()
    df = df.sort_values(by=['Track Point Time'])
    df['Diff'] = df['Track Point Time'].shift(-1) - df['Track Point Time']
    df['DiffSeconds'] = df['Diff'].dt.total_seconds()
    df['DiffSecondsClipped'] = df['DiffSeconds']
    df['DiffSecondsClipped'] = np.clip(df['DiffSeconds'], a_max=900, a_min=None)
    return df

In [ ]:
def trackHistAnalysis(Lat1, Lat2, Long1, Long2, Alt, Title):
    df2005Scope = trackDiffs(trackScope(df[df['Dataset']=='2005-08'],Lat1, Lat2, Long1, Long2, Alt))
    df2010Scope = trackDiffs(trackScope(df[df['Dataset']=='2010-08'],Lat1, Lat2, Long1, Long2, Alt))
    df2015Scope = trackDiffs(trackScope(df[df['Dataset']=='2015-08'],Lat1, Lat2, Long1, Long2, Alt))
    df2018Scope = trackDiffs(trackScope(df[df['Dataset']=='2018-08'],Lat1, Lat2, Long1, Long2, Alt))
    dataScope = [df2005Scope["DiffSecondsClipped"], df2010Scope["DiffSecondsClipped"], df2015Scope["DiffSecondsClipped"], df2018Scope["DiffSecondsClipped"]]

    headers = ["2005", "2010", "2015", "2018"]
    dataScope = pd.concat(dataScope, axis=1, keys=headers)
    
    MainMap = df[df['Dataset']=='2018-08']
    MainMap['Color'] = 'DarkBlue'

    FocusMap = trackScope(df[df['Dataset']=='2018-08'],Lat1, Lat2, Long1, Long2, Alt)
    FocusMap['Color'] = 'Red'

    MapData = pd.concat([MainMap, FocusMap])
    
    MapData.plot.scatter(x='Longitude',
                y='Latitude',
                c=MapData['Color'],
                xlim=(-73.77,-73.65), ylim=(41.000,41.120),
                alpha=0.05, s=3, figsize=(10,10), title=Title);
    
    dataScope[["2005", "2010", "2015", "2018"]].plot.hist(bins=60, alpha=0.2, title="Histogram of Flight Frequency by Seconds", density=True) 
    
    dataScope.hist(color="k", alpha=0.5, bins=60, figsize=(10, 10), density=True);

    return dataScope

# ETL Data

In [ ]:
df2018 = pd.read_csv('data/August 2018 HPN traffic.raw.csv')
df2018 = trackPrep(df2018)

In [ ]:
df2010 = pd.read_csv('data/August 2010 HPN traffic smoothed.csv')
df2010 = trackPrep(df2010)

In [ ]:
df2005 = pd.read_csv('data/August 2005 HPN traffic smoothed.csv')
df2005 = trackPrep(df2005)

In [ ]:
df2015 = pd.read_csv('data/August 2015 HPN traffic.raw.csv')
df2015 = trackPrep(df2015)

In [ ]:
df2005['Dataset'] = '2005-08'
df2010['Dataset'] = '2010-08'
df2015['Dataset'] = '2015-08'
df2018['Dataset'] = '2018-08'
df = pd.concat([df2005, df2010, df2015, df2018])

# Analysis

## Set 1 - Approach End of 34

In [ ]:
set1 = trackHistAnalysis(41.051,41.054,-73.702,-73.692,3500,"KHPN Traffic Frequency Analysis - Approach End of 34")

In [ ]:
set1.describe()

## Set 2 - HPN7 Departure off Runway 16 (Purchase St. and Anderson Hill Road)

In [ ]:
set2 = trackHistAnalysis(41.031,41.041,-73.715,-73.701,3500,"KHPN Traffic Frequency Analysis - HPN7 Departure off 16 in Purchase")

In [ ]:
set2.describe()

## Set 3 - HPN7 Approach End of 16

In [ ]:
set3 = trackHistAnalysis(41.077,41.085,-73.728,-73.708,3500,"KHPN Traffic Frequency Analysis - HPN7 Departure off 16 in Purchase")

In [ ]:
set3.describe()

## Set 4 - Runway 34 Arrivals

In [ ]:
set4 = trackHistAnalysis(41.01,41.02,-73.67,-73.66,3500,"KHPN Traffic Frequency Analysis - Runway 34 Approach")

In [ ]:
set4.describe()

## Set 5 - Runway 16 Arrivals

In [ ]:
set5 = trackHistAnalysis(41.10,41.11,-73.74,-73.73,3500,"KHPN Traffic Frequency Analysis - Runway 16 Approach")

In [ ]:
set5.describe()

## Set 6 - Runway 29 Arrivals

In [ ]:
set6 = trackHistAnalysis(41.05,41.07,-73.69,-73.68,3500,"KHPN Traffic Frequency Analysis - Runway 29 Approach")

In [ ]:
set6.describe()